# Алгоритмы анализа данных

## Тема 3. Классификация. Логистическая регрессия

#### Задание 1. 
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

Сгенерируем датасет

In [18]:
train = pd.read_csv('./train.csv')

In [21]:
X = train.drop(['choose', 'Id'], axis=1)

y = train['choose']

In [23]:
# Функция масштабирования признаков методом стандартизации

def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st.iloc[1, :] = calc_std_feat(X.iloc[1, :])
X_st.iloc[2, :] = calc_std_feat(X.iloc[2, :])
X_st.iloc[3, :] = calc_std_feat(X.iloc[3, :])

In [24]:
# Исходная функция calc_logloss

def calc_logloss(y, y_pred):
    
    err = -np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
   
    return err

In [25]:
# Измененная функция

def calc_logloss(y, y_pred):
    tol = 1e-5
    y_pred = y_pred.copy()
    y_pred = np.clip(y_pred, a_min=tol, a_max=1-tol) # clip - ограничивает прогнозы
    
    score = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
   
    return -score

#### Задание 2. 
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [26]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [27]:
def eval_model(X, y, iterations, alpha=1e-4, tol=1e-5): # Функция обучения модели
    
    np.random.seed(23)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    errors, weights = [], []
    
    for n_iter in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        current_error = calc_logloss(y, y_pred)
        W = W - alpha * (1/n * np.dot((y_pred - y), X.T))
        
        errors.append(current_error)
        weights.append(W)
        
        if n_iter > 2 and np.abs(current_error - errors[-2]) < tol: # Критерий остановки
            break
            
    errors = np.array(errors)
    return errors, weights

#### Задание 3.
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [59]:
def calc_pred_proba(W: np.array, X: np.array) -> np.array:
    """
    Применение обученной модели логистической регрессии
    для матрицы признаков x, получение прогнозов модели.
    
    Parameters
    ----------
    W: np.array
        Веса обученной модели
        
    X: np.array
        Матрица признаков
        
    Returns
    -------
    y_pred: np.array
        Вектор прогнозов.
    
    """
    y_pred = 1 / (1 + np.exp(-np.dot(W, X)))
    
    return y_pred

#### Задание 4.
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [55]:
def calc_pred(W, X, threshold: float=0.5) -> np.array:
    """
    Применение обученной модели логистической регрессии
    для матрицы признаков X, получение лейблов классов.
    
    Parameters
    ----------
    W: np.array
        Веса обученной модели.
        
    X: np.array
        Матрица признаков
        
    threhold: float, optional, default = 0.5.
        Порог, при котором объект относится к целевому классу.
        Опциональный параметр, по умолчанию равен 0.5.
        
    Returns
    -------
    y_pred: np.array
        Вектор прогнозов.
    
    """
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > threshold, 1, 0)
    
    return y_pred

#### Задание 5.
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [13]:
def accuracy_score(y_true: np.array, y_pred: np.array) -> float:
    """
    Вычисление значения метрики Accuracy.
    
    Parameters
    ----------
    y_true: np.array
        Вектор истинных ответов
    
    y_pred: np.array
        Вектор прогнозов.
        
    Returns
    -------
    score: float
        Значение метрики Accuracy
    
    """
    mask = y_true == y_pred
    
    return sum(mask) / y_true.shape[0]

def confusion_matrix(y_true: np.array, y_pred: np.array):
    """
    Вычисление матрицы ошибок.
    Возвращение значения при плоском векторе: tn, fp, fn, tp.
    
    Parameters
    ----------
    y_true: np.array
        Вектор истинных ответов
    
    y_pred: np.array
        Вектор прогнозов
        
    Returns
    -------
    conf_matrix: np.array
        Матрица ошибок.
    
    """
    conf_matrix = np.zeros(shape=(2, 2))
    conf_matrix[0, 0] = np.sum(np.where(y_true == 0, y_pred == y_true, False))
    conf_matrix[1, 1] = np.sum(np.where(y_true == 1, y_pred == y_true, False))
    conf_matrix[1, 0] = np.sum(np.where(y_true == 1, y_pred != y_true, False))
    conf_matrix[0, 1] = len(y_true) - conf_matrix.sum()
    
    return conf_matrix.astype(int)

def precision_score(y_true: np.array, y_pred: np.array) -> float:
    """
    Вычисление значения метрики precision.
    # конверсия TP / (TP + FP)
    
    Parameters
    ----------
    y_true: np.array
        Вектор истинных ответов
    
    y_pred: np.array
        Вектор прогнозов
        
    Returns
    -------
    score: float
        Значение метрики precision.
    
    """
    _, fp, _, tp = confusion_matrix(y_true, y_pred).flatten()
    score = tp / (tp + fp)
    
    return score

def recall_score(y_true: np.array, y_pred: np.array) -> float:
    """
    Вычисление значения метрики recall.
    # Доля ЦА TP / (TP + FN)
    
    Parameters
    ----------
    y_true: np.array
        Вектор истинных ответов
    
    y_pred: np.array
        Вектор прогнозов
        
    Returns
    -------
    score: float
        Значение метрики recall.
    
    """
    _, _, fn, tp = confusion_matrix(y_true, y_pred).flatten()
    score = tp / (tp + fn)
    
    return score

def f1_score(y_true: np.array, y_pred: np.array, betta: float=0.5) -> float:
    """
    Вычисление значения метрики F1.
    
    Parameters
    ----------
    y_true: np.array
        Вектор истинных ответов
    
    y_pred: np.array
        Вектор прогнозов
        
    betta: float
        Вес для регулирования важности между метриками precision / recall.
        Опциональный параметр, по умолчанию равен 0.5.
        
    Returns
    -------
    score: float
        Значение метрики F1.
    
    """
    precision, recall = precision_score(y_true, y_pred), recall_score(y_true, y_pred)
    score = 2 * precision * recall / (precision + recall)
    
    return score

#### Задание 6.
Могла ли модель переобучиться? Почему?

Да, модель могла переобучиться, т.к. очень маленьких набор данных

#### Задание 7. 
Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризацией соответственно.

In [71]:
def eval_model_with_regularization(X: np.array,
                                   y: np.array, 
                                   iterations: int = 100000, 
                                   alpha: float = 0.01, 
                                   tol: float = 1e-5,
                                   c1: float = 0., # коэф. L1-регуляризации
                                   c2: float = 0.): # коэф. L2-регуляризации
    """
    Модель логистической регресси с L1-регуляризацией и L2-регуляризацией.
    
    Parameters
    ----------
    X: np.array
        Матрица признаков для обучения
    
    y: np.array
        Вектор целевой переменной для обучения
    
    iterations:int, optional, default = 100000
        Количество итераций обучения. Опциональный параметр,
        по умолчанию равен 100000.
    
    alpha: float, optional, default = 0.01
        Скорость обучения. Опциональный параметр,
        по умолчанию равен 0.01
    
    tol: float, optional, default = 1e-5
        Минимальное значение изменение ошибки, при котором
        обучение продолжается. Если изменение ошибки на соседних 
        итерациях меньше указанной величины, то обучение прекращается.
    
    c1: float, optional, default = 0
        Коэффициент L1-регуляризации.
        Опциональный параметр, по умолчанию равен 0.
    
    c2: float = 0
        Коэффициент L2-регуляризации.
        Опциональный параметр, по умолчанию равен 0.
    
    """

    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    errors, weights = [], []
    
    for n_iter in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        current_error = calc_logloss(y, y_pred)
        W = W - alpha * (1/n * np.dot((y_pred - y), X.T)) + 2 * c2 * W + 0.5 * c1 * W/np.abs(W)
        
        errors.append(current_error)
        weights.append(W)
        
        if n_iter > 2 and np.abs(current_error - errors[-2]) < tol: # Критерий остановки
            break
            
    errors = np.array(errors)
    return errors, weights